# Weaviate

This notebook covers how to get started with the Weaviate vector store.

>[Weaviate](https://weaviate.io/) is an open-source vector database. It allows you to store data objects and vector embeddings from your favorite ML-models, and scale seamlessly into billions of data objects.

This notebook shows how to use the functionality related to the `Weaviate` vector database.

`Weaviate` can be deployed in many different ways depending on your requirements. For example, you can either connect to a [Weaviate Cloud Services](https://console.weaviate.cloud) instance or a [local Docker instance](https://weaviate.io/developers/weaviate/installation/docker-compose). 
See the `Weaviate` [installation instructions](https://weaviate.io/developers/weaviate/installation) for more information.

## Installation

In [1]:
# install package
%pip install -U langchain-weaviate


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Environment Setup

This example assumes you have a local weaviate instance setup and running on at `http://localhost:8080`.



We want to use `OpenAIEmbeddings` so we have to an Open AI API key and store it in the environment variable `OPENAI_API_KEY`.


## Usage

## Similarity Search

Below you can see a minimal example of how to approach a simple similarity search.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain_weaviate.vectorstores import WeaviateVectorStore

In [3]:

loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

In [4]:
db = WeaviateVectorStore.from_documents(docs, embeddings, weaviate_url="http://localhost:8080", by_text=False)

/workspaces/langchain-weaviate/.venv/lib/python3.12/site-packages/langchain_community/embeddings/openai.py:499: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  response = response.dict()
/workspaces/langchain-weaviate/.venv/lib/python3.12/site-packages/pydantic/main.py:979: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', DeprecationWarning)


In [5]:
query = "What did the president say about Ketanji Brown Jackson"
docs = db.similarity_search(query)

In [6]:
print(docs[0].page_content)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.


## Authentication

Weaviate instances have authentication enabled by default. You can use either a username/password combination or API key.  

In [ ]:
import weaviate

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

# client = weaviate.Client(
#     url=WEAVIATE_URL,
#     auth_client_secret=weaviate.AuthClientPassword(
#         username = "WCS_USERNAME",  # Replace w/ your WCS username
#         password = "WCS_PASSWORD",  # Replace w/ your WCS password
#     ),
# )

vectorstore = Weaviate.from_documents(
    documents, embeddings, client=client, by_text=False
)

## Similarity search with score

Sometimes we might want to perform the search, but also obtain a relevancy score to know how good is a particular result. 
The returned distance score is cosine distance. Therefore, a lower score is better.

docs = db.similarity_search_with_score(query, by_text=False)
docs[0]